In [1]:
import pandas as pd
import numpy as np
from functools import partial
from ctcv_helper import print_table
from ctcv_helper import read_dataframe
from ctcv_helper import read_dataframe_ratio

In [10]:
# Read limits from csv files
df_limits = pd.DataFrame.from_csv("Mar28_unblinded_rangefix/limits_cv_0p5.csv", sep=",", index_col=None)
df_limits = df_limits.append(pd.DataFrame.from_csv("Mar28_unblinded_rangefix/limits_cv_1p0.csv", sep=",", index_col=None), ignore_index=True)
df_limits = df_limits.append(pd.DataFrame.from_csv("Mar28_unblinded_rangefix/limits_cv_1p5.csv", sep=",", index_col=None), ignore_index=True)

# Add the SM-like expected limits
df_limits_exp = pd.DataFrame.from_csv("limits_K6_SM_expected_cv_1p0.csv", sep=",", index_col=None)
df_limits_exp = df_limits_exp.append(pd.DataFrame.from_csv("limits_K6_SM_expected_cv_0p5.csv", sep=",", index_col=None), ignore_index=True)
df_limits_exp = df_limits_exp.append(pd.DataFrame.from_csv("limits_K6_SM_expected_cv_1p5.csv", sep=",", index_col=None), ignore_index=True)

In [11]:
# Read the cross sections from csv files
df_xsbr = {}
df_br = {}
df_xsecs = {}
for model in ['K4', 'K5', 'K6', 'K7']:
    df_xsbr[model]  = pd.DataFrame.from_csv("xsbr_scalings_%s.csv"%model, sep=",", index_col=None)
    df_br[model]    = pd.DataFrame.from_csv("br_scalings_%s.csv"%model,   sep=",", index_col=None)
    df_xsecs[model] = pd.DataFrame.from_csv("xsecs_tH_ttH_%s.csv"%model, sep=",", index_col=None)

In [16]:
# Print pure limits on r
MOD = 'K6'
print " alpha  Ct/CV           CV=0.5                   CV=1.0                   CV=1.5"
print "                  exp  (expSM)   (obs)     exp  (expSM)   (obs)     exp  (expSM)   (obs)"
print_table([partial(read_dataframe, df=df_limits, att='exp'),
             partial(read_dataframe, df=df_limits_exp, att='exp'),
             partial(read_dataframe, df=df_limits, att='obs')],
             linepat=" %6.3f (%6.3f) (%6.3f)")

 alpha  Ct/CV           CV=0.5                   CV=1.0                   CV=1.5
                  exp  (expSM)   (obs)     exp  (expSM)   (obs)     exp  (expSM)   (obs)
-0.973 -6.000:   0.015 ( 0.023) ( 0.026)                                                  
-0.941 -4.000:   0.038 ( 0.058) ( 0.070)                                                  
-0.900 -3.000:   0.076 ( 0.107) ( 0.144)  0.076 ( 0.109) ( 0.144)                         
-0.862 -2.500:   0.117 ( 0.175) ( 0.227)                                                  
-0.800 -2.000:   0.194 ( 0.294) ( 0.385)  0.194 ( 0.282) ( 0.385)  0.194 ( 0.292) ( 0.385)
-0.692 -1.500:   0.352 ( 0.560) ( 0.715)  0.352 ( 0.486) ( 0.715)                         
-0.640 -1.333:                                                     0.443 ( 0.665) ( 0.899)
-0.610 -1.250:                            0.494 ( 0.713) ( 1.004)                         
-0.500 -1.000:   0.709 ( 0.899) ( 1.437)  0.709 ( 0.967) ( 1.437)  0.709 ( 1.036) ( 1.437)
-0.410 -0.8

In [21]:
# Print limit * cross section * BR
def xs_limit(ct,cv, df_xsecs, df_limits, limval='exp'):
    # Take the cross section for cv=1.0, ct=ct/cv
    xsec = read_dataframe(round(ct/cv,3), 1.0, df=df_xsecs, att='tot')
    lim  = read_dataframe(ct, cv, df=df_limits, att=limval)
    return float(xsec)*float(lim)

print " alpha  Ct/CV           CV=0.5                   CV=1.0                   CV=1.5"
print "                  exp   (expSM)   (obs)     exp   (expSM)  (obs)     exp   (expSM)  (obs)"
print_table([partial(xs_limit, df_xsecs=df_xsecs[MOD], df_limits=df_limits, limval='exp'),
             partial(xs_limit, df_xsecs=df_xsecs[MOD], df_limits=df_limits_exp, limval='exp'),
             partial(xs_limit, df_xsecs=df_xsecs[MOD], df_limits=df_limits, limval='obs')],
             linepat=" %6.3f (%6.3f) (%6.3f) ")

 alpha  Ct/CV           CV=0.5                   CV=1.0                   CV=1.5
                  exp   (expSM)   (obs)     exp   (expSM)  (obs)     exp   (expSM)  (obs)
-0.973 -6.000:   0.335 ( 0.522) ( 0.606)                                                     
-0.941 -4.000:   0.337 ( 0.520) ( 0.626)                                                     
-0.900 -3.000:   0.335 ( 0.474) ( 0.639)   0.335 ( 0.481) ( 0.639)                           
-0.862 -2.500:   0.334 ( 0.500) ( 0.649)                                                     
-0.800 -2.000:   0.330 ( 0.501) ( 0.656)   0.330 ( 0.480) ( 0.656)   0.330 ( 0.498) ( 0.656) 
-0.692 -1.500:   0.325 ( 0.517) ( 0.660)   0.325 ( 0.449) ( 0.660)                           
-0.640 -1.333:                                                       0.325 ( 0.487) ( 0.659) 
-0.610 -1.250:                             0.321 ( 0.464) ( 0.653)                           
-0.500 -1.000:   0.315 ( 0.399) ( 0.638)   0.315 ( 0.429) ( 0.638)   0.315 ( 

#### Now do this again, but store it in a dataframe and write it to a csv file

In [25]:
df_xs_limits = pd.DataFrame()
df_xs_limits['alpha'] = np.sign(df_limits.cf*df_limits.cv)*df_limits.cf**2/(df_limits.cf**2+df_limits.cv**2)
df_xs_limits['alpha'] = df_xs_limits.alpha.round(4)
df_xs_limits['ratio'] = df_limits.cf/df_limits.cv
df_xs_limits['ratio'] = df_xs_limits.ratio.round(3)

# Use our xs_limit function to store the limits
for limval in ['twosigdown', 'onesigdown', 'exp', 'onesigup', 'twosigup', 'obs']:
    df_xs_limits[limval] = np.vectorize(partial(xs_limit,
                                               df_xsecs=df_xsecs[MOD],
                                               df_limits=df_limits,
                                               limval=limval))(df_limits.cf,
                                                                    df_limits.cv)
# Add the SM expected values
for limval in ['twosigdown', 'onesigdown', 'exp', 'onesigup', 'twosigup']:
    df_xs_limits[limval+'SM'] = np.vectorize(partial(xs_limit,
                                               df_xsecs=df_xsecs[MOD],
                                               df_limits=df_limits_exp,
                                               limval=limval))(df_limits.cf,
                                                                    df_limits.cv)


# Remove the duplicate entries, sort, reset the index, and write to csv
df_xs_limits.drop_duplicates(subset='ratio', inplace=True)
df_xs_limits.sort_values(by='ratio', inplace=True)
df_xs_limits.index = range(1,len(df_xs_limits)+1)
df_xs_limits.to_csv("xs_limits_%s.csv"%MOD)

df_xs_limits

,alpha,ratio,twosigdown,onesigdown,exp,onesigup,twosigup,obs,twosigdownSM,onesigdownSM,expSM,onesigupSM,twosigupSM
1,-0.9730,-6.000,0.178018,0.245064,0.335229,0.471633,0.651963,0.605724,0.261675,0.350705,0.521801,0.702448,0.882413
2,-0.9412,-4.000,0.176217,0.239406,0.337305,0.474363,0.650580,0.625661,0.214204,0.364263,0.519594,0.667578,1.023369
3,-0.9000,-3.000,0.179382,0.239176,0.335290,0.473037,0.650205,0.639132,0.252137,0.340478,0.474123,0.614612,0.846454
4,-0.8621,-2.500,0.177221,0.236962,0.333576,0.473067,0.648573,0.648573,0.217665,0.336140,0.500210,0.700839,1.006983
5,-0.8000,-2.000,0.174902,0.235077,0.330370,0.471348,0.644375,0.655626,0.227099,0.336740,0.501456,0.693170,0.880899
6,-0.6923,-1.500,0.170886,0.230031,0.325255,0.464030,0.642482,0.659645,0.227155,0.347076,0.517131,0.668317,1.011916
7,-0.6400,-1.333,0.169476,0.228412,0.325025,0.463641,0.641913,0.658700,0.215824,0.342081,0.487214,0.644736,1.143239
8,-0.6098,-1.250,0.167619,0.226484,0.321384,0.461034,0.636524,0.653370,0.230530,0.307892,0.463819,0.646883,1.099476
9,-0.5000,-1.000,0.162920,0.221607,0.314742,0.456487,0.634456,0.637785,0.191259,0.290980,0.399199,0.604464,0.793558
10,-0.4098,-0.833,0.159174,0.217191,0.312277,0.450437,0.631772,0.614569,0.226688,0.301789,0.432114,0.629647,0.855236


#### Produce also a csv file with the pure r limits (for cv == 1)

In [23]:
df_r_limits = pd.DataFrame()
df_r_limits['alpha'] = np.sign(df_limits.cf*df_limits.cv)*df_limits.cf**2/(df_limits.cf**2+df_limits.cv**2)
df_r_limits['alpha'] = df_r_limits.alpha.round(4)
df_r_limits['ratio'] = df_limits.cf/df_limits.cv
df_r_limits['ratio'] = df_r_limits.ratio.round(3)

# Use our xs_limit function to store the limits
for limval in ['twosigdown', 'onesigdown', 'exp', 'onesigup', 'twosigup', 'obs']:
    df_r_limits[limval] = getattr(df_limits, limval)

for limval in ['twosigdown', 'onesigdown', 'exp', 'onesigup', 'twosigup']:
    df_r_limits[limval+'SM'] = getattr(df_limits_exp, limval)


# Remove the duplicate entries, sort, reset the index, and write to csv
df_r_limits.drop_duplicates(subset='ratio', inplace=True)
df_r_limits.sort_values(by='ratio', inplace=True)
df_r_limits.index = range(1,len(df_xs_limits)+1)
df_r_limits.to_csv("r_limits_%s.csv"%MOD)


In [24]:
df_r_limits

,alpha,ratio,twosigdown,onesigdown,exp,onesigup,twosigup,obs,twosigdownSM,onesigdownSM,expSM,onesigupSM,twosigupSM
1,-0.9730,-6.000,0.0077,0.0106,0.0145,0.0204,0.0282,0.0262,0.048888,0.068420,0.108690,0.154499,0.200764
2,-0.9412,-4.000,0.0198,0.0269,0.0379,0.0533,0.0731,0.0703,0.150913,0.215343,0.281646,0.380121,0.475100
3,-0.9000,-3.000,0.0405,0.0540,0.0757,0.1068,0.1468,0.1443,0.235051,0.352755,0.486170,0.683010,0.938558
4,-0.8621,-2.500,0.0620,0.0829,0.1167,0.1655,0.2269,0.2269,0.354420,0.473358,0.713082,0.994526,1.690350
5,-0.8000,-2.000,0.1026,0.1379,0.1938,0.2765,0.3780,0.3846,0.573368,0.731115,0.967403,1.599700,2.330790
6,-0.6923,-1.500,0.1852,0.2493,0.3525,0.5029,0.6963,0.7149,0.781753,1.052980,1.475280,2.146580,3.358200
7,-0.6400,-1.333,0.2312,0.3116,0.4434,0.6325,0.8757,0.8986,0.294427,0.466668,0.664659,0.879550,1.559610
8,-0.6098,-1.250,0.2577,0.3482,0.4941,0.7088,0.9786,1.0045,0.076149,0.117597,0.174996,0.245185,0.352288
9,-0.5000,-1.000,0.3670,0.4992,0.7090,1.0283,1.4292,1.4367,0.981897,1.322170,1.790520,2.592620,5.023300
10,-0.4098,-0.833,0.4719,0.6439,0.9258,1.3354,1.8730,1.8220,0.672056,0.894708,1.281080,1.866700,2.535500
